<a href="https://colab.research.google.com/github/TiagoIesbick/dashboard-etl/blob/main/settled_last_month.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [13]:
%%capture
!pip install tabula-py[jpype]
import pandas as pd
import pathlib
import re
from tabula import read_pdf

In [14]:
def str_to_float(value: str) -> float:
  value = str(value).replace('.', '').replace(',', '.')
  value = re.sub(r'[A-Za-z]|\s', '', value)
  return float(0) if value == '' else float(value)

def agrupamento_format(value: str) -> str:
  if re.search(r'^0{2}', value[:2]):
    value = '   ' + value
  elif re.search(r'^[1-9]\d', value[:2]):
    value = '      ' + value
  return value

In [20]:
folder = pathlib.Path(r'/content/drive/MyDrive/settled_last_month')

for pdf_file in folder.glob('*.pdf'):
  fonte = read_pdf(pdf_file, area=[115, 71, 115+20, 71+245], pages=1, pandas_options={'header': None})[0]
  fonte = re.search(r'[\d\.]*', fonte.iloc[0,1]).group()
  df = read_pdf(pdf_file, area=[180, 71, 180+370, 71+755], guess=False, pandas_options={'header': [0]}, pages='all', multiple_tables=False, encoding='latin1')[0]
  df.drop(df.loc[df['Agrupamento']=='Agrupamento'].index, inplace=True)
  df_out = df.loc[df['%'].isna()]
  for i in df_out.index:
    df.loc[i-1, 'Agrupamento'] += ' ' + df.loc[i, 'Agrupamento']
  df.drop(df_out.index, inplace=True)
  if len(df.loc[df['Agrupamento']=='Total']) > 1:
    df.drop(df.loc[df['Agrupamento']=='Total'].index[1:], inplace=True)
  df.reset_index(drop=True, inplace=True)
  df.iloc[:, 1:] = df.iloc[:, 1:].map(str_to_float)
  display(df.loc[~df['Agrupamento'].str.startswith(('T', '0')), ['Atualizado', 'Pré-Empenhado', 'Empenhado', 'Disponível', 'Liquidado', 'Pago', 'A Liquidar', 'A Pagar']].sum())
  df['unidorc'] = int(re.sub(r'\D', '', df.iloc[1,0]))
  df['fonte'] = fonte
  li = list(df.loc[df['Agrupamento'].str.startswith('00')].index)
  li.append(df.index[-1])
  for i,j in enumerate(li):
    subacao = int(re.sub(r'\D', '',df.loc[j, 'Agrupamento']))
    if i+1 < len(li):
      idx = li[i]
      while idx <= li[i+1]:
        df.loc[idx, 'subacao'] = subacao
        idx += 1
  df.loc[~df['Agrupamento'].str.startswith(('0', 'T')), 'elemento'] = df['Agrupamento'].str.replace(r'\D', '', regex=True)
  display(df)
  break

,0
Atualizado,941343.28
Pré-Empenhado,958528.88
Empenhado,2330825.58
Disponível,1804953.09
Liquidado,1327580.8
Pago,511163.36
A Liquidar,1267620.74
A Pagar,1389286.34


,Agrupamento,Dotação Inicial,Atualizado,Pré-Empenhado,Empenhado,Disponível,Liquidado,Pago,A Liquidar,A Pagar,%,unidorc,fonte,subacao,elemento
0,Total,0.0,0.0,958528.88,2330825.58,1801953.19,1327580.8,511163.36,1003244.78,816417.44,0.0,7001,1.802.069.001,NaN,NaN
1,07001 ADMINISTRAÇÃO PREVIMPA,0.0,0.0,958528.88,2330825.58,1801953.19,1327580.8,511163.36,1003244.78,816417.44,0.0,7001,1.802.069.001,NaN,NaN
2,002529 ADMINISTRAÇÃO GERAL - PREVIMPA,0.0,0.0,243645.14,325306.6,10989.82,142215.62,159909.31,183090.98,17693.69,0.0,7001,1.802.069.001,2529.0,NaN
3,33.90.14 Diárias - Civil,0.0,9688.36,0.0,8595.44,1092.92,8595.44,10955.89,0.0,2360.45,88.72,7001,1.802.069.001,2529.0,339014
4,33.90.30 Material De Consumo,0.0,0.0,0.0,1565.24,1565.24,951.5,4105.5,613.74,3154.0,0.0,7001,1.802.069.001,2529.0,339030
5,33.90.33 Passagens E Despesas Com Locomoção,0.0,0.0,0.0,0.0,0.0,233.1,119.88,233.1,113.22,0.0,7001,1.802.069.001,2529.0,339033
6,33.90.35 Serviços De Consultoria,0.0,0.0,0.0,0.0,0.0,14300.0,14300.0,14300.0,0.0,0.0,7001,1.802.069.001,2529.0,339035
7,33.90.37 Locação De Mão-De-Obra,0.0,0.0,0.0,0.0,0.0,30901.4,35934.66,30901.4,5033.26,0.0,7001,1.802.069.001,2529.0,339037
8,33.90.39 Outros Serviços De Terceiros - Pessoa...,0.0,70671.64,24560.0,29640.0,5080.0,54883.1,68158.26,25243.1,13275.16,41.94,7001,1.802.069.001,2529.0,339039
9,33.90.40 Serviços De Tecnologia Da Informação ...,0.0,0.0,0.0,5437.5,5437.5,26774.48,20758.52,21336.98,6015.96,0.0,7001,1.802.069.001,2529.0,339040
